# Método de Broyden para sistemas de ecuaciones no lineales.
## Andrade Vidal Fernando adrian.

### ¿Qué es el metodo de Broyden?

Es un método que tarda más en converger pero tiene menos complejidad en sus cálculos a diferencia del método de Newton, la principal ventaja de este método es que no tenemos que calcular la inversa de la Jacobina, haciendo mucho más sencillo el calculo para algunos sistemas.

#### Características:

+ No se requiere invertir la matriz jacobiana.
+ Es necesario contar con dos aproximaciones a la solución.
+ La convergencía sera más rápida que el punto fijo pero más lenta que en el método de Newton.

El presente cuadernillo proporcionara paso a paso la solución de un sistema de ecuaciones no lineales, aunque puede ser modificado para acoplarlo a tus necesidades.


In [2]:
from sympy import * #Usaremos SymPy para computo simbólico.

import pandas as pd #Usaremos Pandas para crear y manejar data frames donde registraremos los datos.

import IPython.display as disp #Usaremos el display para visualizar ecuaciones.

Estableceremos 3 variables aunque se pueden añadir màs en esta declaración de variables.

In [3]:
#Declaramos los simbolos que usaremos para las ecuaciónes.

x,y,z = symbols('x y z')

#Estamos insertando las funciones con las que trabajaremos, notar que puede haber simplificaciones.

funcion_original_1 = x**2+y-37
funcion_original_2 = x-y**2-5
funcion_original_3 = x+y+z-3

#Creamos nuestra matrix de las funciones insertadas

matriz_funciones = Matrix([funcion_original_1, funcion_original_2, funcion_original_3])

#matriz_funciones = Matrix([funcion_original_1, funcion_original_2])


#Imprimimos las funciones insertadas para asegurarnos.

print("Esta es la primer ecuación original: \n")
disp.display(funcion_original_1)

print("\n\nEsta es la segunda ecuación original: \n")
disp.display(funcion_original_2)

print("\n\nEsta es la tercer ecuación original: \n")
disp.display(funcion_original_3)

print("\n\nEsta es la matrix de nuestras ecuaciones: \n")
display(matriz_funciones)

Esta es la primer ecuación original: 



x**2 + y - 37



Esta es la segunda ecuación original: 



x - y**2 - 5



Esta es la tercer ecuación original: 



x + y + z - 3



Esta es la matrix de nuestras ecuaciones: 



Matrix([
[x**2 + y - 37],
[ x - y**2 - 5],
[x + y + z - 3]])

Encontraremos una segunda aproximación usando el metodo de Newton.

In [62]:
#Obtenemos la jacobiana de la matriz con respecto a los simbolos x,y,z

jacobiano = matriz_funciones.jacobian([x, y, z])

#jacobiano = matriz_funciones.jacobian([x, y])

display(jacobiano)

Matrix([
[2*x,    1, 0],
[  1, -2*y, 0],
[  1,    1, 1]])

In [65]:
# Declaramos nuestro punto inicial y nuestro numero de iteraciones a realizar.

x_i= 0
y_i= 0
z_i= 0

numero_iteraciones = 20

#Data Frame donde guardaremos los resultados.
df = pd.DataFrame()

jacobiano_evaluado = jacobiano.subs({x:x_i,y:y_i,z:z_i})#Evaluamos el jacobiano en el punto dado.
    
jacobiano_evaluado_inverso = jacobiano_evaluado**-1 #Obtenemos la inversa del jacobiano evaluado.
matriz_funciones_evaluado = matriz_funciones.subs({x:x_i,y:y_i,z:z_i}) #Obtenemos nuestras ecuaciones evaluadas.

solucion_sistema = jacobiano_evaluado_inverso * matriz_funciones_evaluado #Resolvemos el sistema de ecuaciones.
    
matriz_solucion = Matrix([x_i, y_i, z_i]) #Convertimos a matriz nuestras soluciones
    
nueva_solucion_matriz = matriz_solucion - solucion_sistema #Resolvemos las ultimas operaciones para dar el nuevo punto
    
#Esta sera la nueva solucion de la matriz la cual estamos asignando de nuevo a las variables para repetir el proceso
    
#Hay que notar que estamos convirtiendo los numeros a flotantes de nuevo para que sea mas facil trabajar con ellos y rápido.
    
x_ii = float(nueva_solucion_matriz[0,0])
y_ii = float(nueva_solucion_matriz[1,0])
z_ii = float(nueva_solucion_matriz[2,0])


Ahora empezamos a utilizar el metodo de Broyden en este punto porque ya aplicamos newton para encontrar otra aproximación.

In [66]:
iteraciones = 0

while iteraciones <= numero_iteraciones:
    
    #Empezamos obteniendo el valor de s
    si = Matrix([x_i,y_i,z_i]) - Matrix([x_ii, y_ii, z_ii])


    #Ahora encontramos Yi

    yi = matriz_funciones.subs({x:x_i,y:y_i,z:z_i}) - matriz_funciones.subs({x:x_ii,y:y_ii,z:z_ii})


    #obtenemos nuestros parametros necesarios

    p = Transpose(si) * jacobiano_evaluado_inverso

    u = si - (jacobiano_evaluado_inverso * yi)

    v = u*p

    w = Transpose(si) * jacobiano_evaluado_inverso * yi

    jacobiano_evaluado_inverso = jacobiano_evaluado_inverso + (v / w[0,0])

    nueva_solucion_broy = Matrix([x_ii,y_ii,z_ii]) -(jacobiano_evaluado_inverso * matriz_funciones.subs({x:x_ii,y:y_ii,z:z_ii}))



    x_i = x_ii
    y_i = y_ii
    z_i = z_ii

    x_ii = nueva_solucion_broy[0,0]
    y_ii = nueva_solucion_broy[1,0]
    z_ii = nueva_solucion_broy[2,0]

    df = df.append({
                'xi':x_i,
                'yi':y_i,
                'zi':z_i,
                'xii':x_ii,
                'yii':y_ii,
                'zii':z_ii,
                'f1_original':float(funcion_original_1.subs({x:x_ii,y:y_ii,z:z_ii})), #Se evalua la funcion 1 original en el punto dado
                'f2_original':float(funcion_original_2.subs({x:x_ii,y:y_ii,z:z_ii})), #Se evalua la funcion 2 original en el punto dado
                'f3_original':float(funcion_original_3.subs({x:x_ii,y:y_ii,z:z_ii}))}, #Se evalua la funcion 3 original en el punto dado
                ignore_index=True #Ignorar este parametro.
                 )

    iteraciones += 1
    
disp.display(df)







,f1_original,f2_original,f3_original,xi,xii,yi,yii,zi,zii
0,4491.095936,-1540.040055,-1.705303e-13,5.0,-67.0058281156964,37.0,38.3149347720178,-39.0,31.6908933436784
1,827.719152,-213.976339,1.762146e-12,-67.0058281156964,29.6676129402959,38.3149347720178,-15.4481051100166,31.6908933436784,-11.2195078302775
2,2402.979830,-539.040778,3.907985e-14,29.6676129402959,49.6400978911455,-15.4481051100166,-24.1594883148565,-11.2195078302775,-22.4806095762890
3,305.623523,-132.602978,-1.243450e-14,49.6400978911455,18.8341350554732,-24.1594883148565,-12.1011203412355,-22.4806095762890,-3.73301471423773
4,135.936253,-146.272584,-4.085621e-14,18.8341350554732,13.6154932507644,-12.1011203412355,-12.4454038617506,-3.73301471423773,1.82991061098611
5,11.746425,-988.051633,-2.344791e-13,13.6154932507644,-8.94187656135184,-12.4454038617506,-31.2107314284221,1.82991061098611,43.1526079897737
6,220.699223,-24.174097,6.394885e-14,-8.94187656135184,16.2373018411103,-31.2107314284221,-5.95074778038912,43.1526079897737,-7.28655406072114
7,185.693324,9.797698,4.796163e-14,16.2373018411103,14.9353380567383,-5.95074778038912,-0.370999303892090,-7.28655406072114,-11.5643387528462
8,75.340243,-85.146029,2.486900e-14,14.9353380567383,10.1409219480840,-0.370999303892090,9.50194457227730,-11.5643387528462,-16.6428665203612
9,278.219225,-793.894359,9.947598e-14,10.1409219480840,18.5373829950884,9.50194457227730,-28.4153434242493,-16.6428665203612,12.8779604291610
